In [1]:
import glob
from afinn import Afinn
pos_review=(glob.glob("c:/data/imdb/train/pos/*.txt"))[20]
f = open(pos_review, 'r')
lines1 = f.readlines()[0]
f.close()
afinn=Afinn()
afinn.score(lines1)

8.0

In [2]:
files=list(glob.glob('c:/data/imdb/train/pos/*.txt')[:10])
files

['c:/data/imdb/train/pos\\0_9.txt',
 'c:/data/imdb/train/pos\\10000_8.txt',
 'c:/data/imdb/train/pos\\10001_10.txt',
 'c:/data/imdb/train/pos\\10002_7.txt',
 'c:/data/imdb/train/pos\\10003_8.txt',
 'c:/data/imdb/train/pos\\10004_8.txt',
 'c:/data/imdb/train/pos\\10005_7.txt',
 'c:/data/imdb/train/pos\\10006_7.txt',
 'c:/data/imdb/train/pos\\10007_7.txt',
 'c:/data/imdb/train/pos\\10008_7.txt']

In [3]:
afinn=Afinn()
for i in files:
    f=open(i)
    lines1=f.readlines()[0]
    print(afinn.score(lines1))
    f.close()

-1.0
2.0
19.0
3.0
14.0
8.0
22.0
28.0
13.0
5.0


In [4]:
neg_review=(glob.glob("c:/data/imdb/train/neg/*.txt"))[20]
f=open(neg_review,'r')
lines2=f.readlines()[0]
f.close()
afinn.score(lines2)

-4.0

In [5]:
files=list(glob.glob('c:/data/imdb/train/neg/*.txt')[:10])
files

['c:/data/imdb/train/neg\\0_3.txt',
 'c:/data/imdb/train/neg\\10000_4.txt',
 'c:/data/imdb/train/neg\\10001_4.txt',
 'c:/data/imdb/train/neg\\10002_1.txt',
 'c:/data/imdb/train/neg\\10003_1.txt',
 'c:/data/imdb/train/neg\\10004_3.txt',
 'c:/data/imdb/train/neg\\10005_3.txt',
 'c:/data/imdb/train/neg\\10006_4.txt',
 'c:/data/imdb/train/neg\\10007_1.txt',
 'c:/data/imdb/train/neg\\10008_2.txt']

In [6]:
afinn=Afinn()
for i in files:
    f=open(i) 
    lines1=f.readlines()[0]
    print(afinn.score(lines1))
    f.close()


6.0
-4.0
9.0
5.0
-7.0
1.0
13.0
4.0
7.0
6.0


In [7]:
pos_review=(glob.glob("c:/data/imdb/train/pos/*.txt")[:100])

lines_pos=[]
for i in pos_review:
    try:
        f=open(i,'r')
        temp=f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except Exception as e:
        continue

len(lines_pos)


100

In [8]:
neg_review=(glob.glob("c:/data/imdb/train/neg/*.txt")[:100])

lines_neg=[]
for i in neg_review:
    try:
        f=open(i,'r')
        temp=f.readlines()[0]
        lines_neg.append(temp)
        f.close()
    except Exception as e:
        continue

len(lines_neg)



100

In [9]:
total_text=lines_pos+lines_neg
len(total_text)

200

In [10]:
import numpy as np
from nltk.corpus import stopwords
x=np.array(["pos","neg"])
class_Index=np.repeat(x, [len(lines_pos),len(lines_neg)],axis=0)

stop_words=stopwords.words('english')

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect=TfidfVectorizer(stop_words=stop_words).fit(total_text)
X_train_vectorized=vect.transform(total_text)
X_train_vectorized.index=class_Index

In [12]:
import pandas as pd
df=pd.DataFrame(X_train_vectorized.toarray(),columns=vect.vocabulary_.keys())
df.head()

,bromwell,high,cartoon,comedy,ran,time,programs,school,life,teachers,...,zombified,auteur,ample,opportunities,golden,geist,uttered,downloading,midget,tricking
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression(random_state=10)
logit.fit(X_train_vectorized,class_Index)

LogisticRegression(random_state=10)

In [14]:
def pos_review(model):
    count_all=0
    count=0
    num=100
    tests1=[]
    for idx in range(0,num):
        pos_review_test=(glob.glob("c:/data/imdb/test/pos/*.txt"))[idx]
        f=open(pos_review_test, 'r',encoding="utf-8")
        tests1.append(f.readlines())
        f.close()

    for test in tests1:
        preds=model.predict(vect.transform(test))
        result=preds[0]
        if result=="pos":
            count+=1
        count_all += 1
    rate=count*100/count_all
    print("예측정확도:{0:.1f}%".format(rate))

In [15]:
def neg_review(model):
    count_all=0
    count=0
    num=100
    tests2=[]
    for idx in range(0,num):
        pos_review_test=(glob.glob("c:/data/imdb/test/neg/*.txt"))[idx]
        f=open(pos_review_test, 'r',encoding="utf-8")
        tests2.append(f.readlines())
        f.close()

    for test in tests2:
        preds=model.predict(vect.transform(test))
        result=preds[0]
        if result=="neg":
            count+=1
        count_all += 1
    rate=count*100/count_all
    print("예측정확도:{0:.1f}%".format(rate))

In [16]:
pos_review(logit)
neg_review(logit)

예측정확도:66.0%
예측정확도:81.0%


In [17]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(random_state=10)
tree.fit(X_train_vectorized, class_Index)

DecisionTreeClassifier(random_state=10)

In [18]:
pos_review(tree)
neg_review(tree)

예측정확도:39.0%
예측정확도:66.0%


In [19]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=10,random_state=10)
forest.fit(X_train_vectorized,class_Index)

RandomForestClassifier(n_estimators=10, random_state=10)

In [20]:
pos_review(forest)
neg_review(forest)

예측정확도:44.0%
예측정확도:75.0%


In [21]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train_vectorized,class_Index)

KNeighborsClassifier(n_neighbors=2)

In [22]:
pos_review(knn)
neg_review(knn)

예측정확도:34.0%
예측정확도:85.0%


In [23]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=10)
mlp.fit(X_train_vectorized,class_Index)

MLPClassifier(random_state=10)

In [24]:
pos_review(mlp)
neg_review(mlp)

예측정확도:58.0%
예측정확도:79.0%


In [25]:
from sklearn.svm import SVC
svm=SVC(random_state=10)
svm.fit(X_train_vectorized,class_Index)

SVC(random_state=10)

In [26]:
pos_review(svm)
neg_review(svm)

예측정확도:63.0%
예측정확도:87.0%


In [27]:
from textblob import TextBlob
a=TextBlob("파이썬은 재미있다.")
a.detect_language()

'ko'

In [28]:
b=TextBlob("I like python.")
b.detect_language()

'en'

In [29]:
c=TextBlob("파이썬은 재미있다")
c.translate(to="en")

TextBlob("Python is fun")

In [30]:
d=TextBlob("I like python")
d.translate(to="ko")

TextBlob("나는 파이썬을 좋아한다")

In [33]:
from textblob.classifiers import NaiveBayesClassifier
train=[
    ('I love this sandwich.','pos'),
    ('This is an amazing place!','pos'),
    ('I feel very good about these beers.','pos'),
    ('This is my best word.','pos'),
    ('What an awesome view','pos'),
    ('I do not like this restaurant','neg'),
    ('I am tired of this stuff.','neg'),
    ("I can't deal with this",'neg'),
    ('He is my sworn enemy!','neg'),
    ('My boss is horrible','neg')
]
test=[
    ('The beer was good.','pos'),
    ('I do not enjoy my job','neg'),
    ("I ain't feeling dandy today.",'eng'),
    ("I feel amazing!", 'pos'),
    ('Gary is a friend of mine.', 'pos'),
    ("I can't believe I'm doing this.", 'neg')
]

In [34]:
cl=NaiveBayesClassifier(train)

In [35]:
print(cl.classify("Their burgers are amazing"))
print(cl.classify("I don't like their pizza."))

pos
neg


In [38]:
blob = TextBlob("The beer was amazing. But the hangover was horrible. My boss was not happy.", classifier=cl)
blob.classify()

'neg'

In [39]:
for sentence in blob.sentences:
    print(sentence,'==>',sentence.classify())

The beer was amazing. ==> pos
But the hangover was horrible. ==> neg
My boss was not happy. ==> neg


In [40]:
for row in test:
    print(row[0],'==>',cl.classify(row[0]))

The beer was good. ==> pos
I do not enjoy my job ==> neg
I ain't feeling dandy today. ==> neg
I feel amazing! ==> pos
Gary is a friend of mine. ==> neg
I can't believe I'm doing this. ==> neg


In [41]:
cl.accuracy(test)

0.6666666666666666

In [42]:
cl.show_informative_features(5)

Most Informative Features
          contains(this) = True              neg : pos    =      2.3 : 1.0
          contains(this) = False             pos : neg    =      1.8 : 1.0
          contains(This) = False             neg : pos    =      1.6 : 1.0
            contains(an) = False             neg : pos    =      1.6 : 1.0
             contains(I) = False             pos : neg    =      1.4 : 1.0
